#### Title:          alphapose_annotate_data.ipynb
#### Description:    Merge output files to a single CSV file per video
#### Author:         Fabian Kahl

In [ ]:
from numpy import load
import pandas as pd
import cv2
import os
import json
import numpy as np

In [ ]:
def json_to_csv(json_path, video, csv_dir_out):
    data = json.load(open(json_path))
    duration = np.nan

    # Source: https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/output.md
    joint_names = ['Nose', 'Neck', 'RShoulder', 'RElbow',
                   'RWrist', 'LShoulder', 'LElbow',
                   'LWrist', 'RHip', 'RKnee', 'RAnkle',
                   'LHip', 'LKnee', 'LAnkle',
                   'REye', 'LEye', 'REar', 'LEar']
    dims = ['X', 'Y', 'C']
    cols = []
    for joint in joint_names:
        for suffix in dims:
            cols.append(joint + suffix)

    # Get timestamp and frame
    cap = cv2.VideoCapture(video)
    timestamps = []
    while cap.isOpened():
        success, image = cap.read()
        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
        if not success:
            print("Ignoring empty video frame.")
            break
        timestamps.append(timestamp_ms)
    cap.release()

    df = pd.DataFrame()
    for k in range(0, len(timestamps)):
        try:
            df_frame_stacked = pd.DataFrame([data[str(k)+'.jpg']['people'][0]['pose_keypoints_2d']], columns=cols)
        except:
            df_frame_stacked = pd.DataFrame([[np.nan]*54], columns=cols)

        # Append df_frame_stacked on df containing all frames
        df = pd.concat([df, df_frame_stacked])

    path_split = video.rsplit('/', 1)
    vid_name_in = path_split[-1]
    csv_name_out = vid_name_in.replace('.mp4', '.csv')

    # Fill some meta data
    vid_in_split = vid_name_in.split('_')
    df['Subject'] = 'Sub'+str(int(vid_in_split[0]))
    df['exerciseName'] = vid_in_split[2]
    df['ipad_view'] = vid_in_split[3].split('.')[0]
    df['duration'] = duration
    df['Timestamp'] = timestamps
    df['Frame'] = list(range(1, len(df)+1))
    # Rearrange the cols to have the meta data in front
    cols = df.columns.to_list()
    df = df[cols[-6:] + cols[:-6]] # Move last 6 cols to the front

    if not os.path.exists(csv_dir_out):
        os.mkdir(csv_dir_out)
    df.to_csv(csv_dir_out + csv_name_out, index=False)

def list_subdirs(folder_path):
    subdirs = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    return subdirs

In [ ]:
results_dir = '../data/DZNE_runs/AlphaPose/'
subdirs = list_subdirs(results_dir)
for subdir in subdirs:
    subsubdirs = list_subdirs(results_dir+subdir+'/')
    for subsubdir in subsubdirs:
        json_path = results_dir + subdir + '/' + subsubdir + '/alphapose-results.json'
        video = '../data/vid_cut/' + subdir + '/' + subsubdir.replace('.MOV', '.mp4')
        csv_dir_out = '../data/non_area_fileformat/frameworks_orig/AlphaPose/' + subdir + '/'
        if not os.path.exists(csv_dir_out):
            os.makedirs(csv_dir_out)
        json_to_csv(json_path, video, csv_dir_out)